In [1]:
#Imports
import os
import tensorflow as tf

import sys
sys.path.insert(0, '..')
from utils.federatedAggregation import FederatedAggregation
from utils.reinforcementLearningHelper import *
from utils.federatedLearningHelper import *

In [2]:
#Setup Environments of selected buildings for training, evaluation, and testing

environments, observation_spec, action_spec  = setup_energymanagement_environments2(num_buildings=30, ecoPriority=1, noise_scale=0)

#Check environment setup
print(
    "Batch size:", environments["train"][f"building_1"].batch_size, 
    "/ State Space: {} / Action Space: {}".format(observation_spec.shape[0], action_spec.shape[0]),
    "/ Upper bound: {}".format(round(environments["train"][f"building_1"].action_spec().maximum.item(), 3)),
)

Batch size: 1 / State Space: 58 / Action Space: 1 / Upper bound: 2.3


In [3]:
# Cluster similar buildings (K-Means with DTW)

#Set parameter
num_clusters = 9 # 2, 6, 10, 12, 14, 16, 18

clustered_buildings = prosumption_clustered_buildings(num_clusters)
clustered_buildings

{0: array([7], dtype=int64),
 1: array([ 3,  4,  5,  9, 10, 11, 16, 23, 25, 26, 27, 30], dtype=int64),
 2: array([1], dtype=int64),
 3: array([24], dtype=int64),
 4: array([12, 13, 14, 15, 19, 20, 22, 28, 29], dtype=int64),
 5: array([ 6, 17], dtype=int64),
 6: array([8], dtype=int64),
 7: array([ 2, 18], dtype=int64),
 8: array([21], dtype=int64)}

In [4]:
# Setup Agent networks
federated_rounds = 3
batch_size = 128
replay_buffer_capacity = 20000 #-> only <18.000 samples per dataset
initial_collect_steps = 2000 #2000
collect_steps_per_iteration = 30 
num_iterations = 5000 #10000
eval_interval = num_iterations-1 #9999

In [5]:
csv_name = "NEW_FL_PPO_OnlyEmissions_v5"

In [6]:
# FEDERATED LEARNING - Initalization Round 0

tf.compat.v1.reset_default_graph()
global_step = tf.compat.v1.train.get_or_create_global_step()

#Initalize a global model for each Cluster of similar buildings
for cluster in range(num_clusters):
        
        # 1. Build global agent per cluster
        first_building_in_cluster = clustered_buildings[cluster][0]
        global_ppo_agent, global_eval_policy, global_collect_policy = initialize_ppo_agent(
                observation_spec=observation_spec, action_spec=action_spec, global_step=global_step, environments=environments,
                )

        # 2. Initially store weights
        model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster}/FLround{0}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        
        save_ppo_weights(global_ppo_agent, model_dir)

In [7]:
#FEDERATED LEARNING - Model training for multiple Rounds

#For each federated round and cluster
for federated_round  in range(federated_rounds):
    for cluster_number, buildings_in_cluster in clustered_buildings.items():

        #Iterate through the buildings per cluster
        print(f"Cluster {cluster_number}: Buildings {buildings_in_cluster} Federated round ---", federated_round+1, f"/ {federated_rounds}")
        local_storage = {
            "actor_weights": [], "value_weights": [], "performance_metrics": []
            }
        
        for building_index in buildings_in_cluster:
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()
            
            #1. Initalize local agent
            local_ppo_agent, local_eval_policy, local_collect_policy = initialize_ppo_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{federated_round}_c{num_clusters}_AvgAgg")
            local_ppo_agent = set_weights_to_ppo_agent(local_ppo_agent, model_dir)
            
            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            local_iterator, local_collect_driver, local_time_step, local_policy_state = setup_rl_training_pipeline(
                local_ppo_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                local_collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Train, evaluate agent and store weights
            local_ppo_agent, local_storage = local_agent_training_and_evaluation(
                local_iterator, local_collect_driver, local_time_step, local_policy_state, global_step, 
                local_ppo_agent, local_eval_policy, local_storage, building_index, num_iterations, environments, agent_type="ppo"
                )           

        # Performe Federated aggregation
        average_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["actor_weights"], local_storage["performance_metrics"])
        average_value_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["value_weights"], local_storage["performance_metrics"]) 
        
        #Save federated weights for next round (Round + 1)
        model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{federated_round+1}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        save_federated_ppo_weights(model_dir, average_actor_weights, average_value_weights)

Cluster 0: Buildings [7] Federated round --- 1 / 3
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Return:  -35878.133
Cluster 1: Buildings [ 3  4  5  9 10 11 16 23 25 26 27 30] Federated round --- 1 / 3
Return:  -50407.164
Return:  -68239.91
Return:  -67271.82
Return:  -68382.83
Return:  -69888.4
Return:  -68969.47
Return:  -71712.625
Return:  -68098.45
Return:  -68319.71
Return:  -70562.24
Return:  -69165.31
Return:  -44757.434
Cluster 2: Buildings [1] Federated round --- 1 / 3
Return:  -70554.75
Cluster 3: Buildings [24] Federated round --- 1 / 3
Return:  -68198.57
Cluster 4: Buildings [12 13 14 15 19 20 22 28 29] Federated round --- 1 / 3
Return:  -68293.77
Return:  -67535.39
Return:  -66560.42
Return:  -40787.434
Return:  -65648.984
Return:  -42468.043
Return:  -67383.484
Return:  -40146.684
Return:  -68080.5
Cluster 5: Buildings [ 6 17] Federated round --- 1 / 3
Return:  -73084.8
Return:  -69911.45
Cluster 6: Buildings [8] Federated round

In [8]:
# LOCAL REFITTING AND EVALUATION

best_federated_round = 3
num_rounds= 3
num_test_iterations = 1

result_df = pd.DataFrame(columns=['Building', 'Total Profit'])

for cluster_number, buildings_in_cluster in clustered_buildings.items():
    for building_index in buildings_in_cluster:
        
        for round in range(num_rounds):
            print("Cluster: ", cluster_number, " - Building: ", building_index, " - round: ", round)
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()

            #1. Initalize local agent
            tf_ppo_agent, eval_policy, collect_policy = initialize_ppo_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ppo/cluster_{cluster_number}/FLround{best_federated_round}_c{num_clusters}_AvgAgg")
            tf_agent = set_weights_to_ppo_agent(tf_ppo_agent, model_dir)

            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            iterator, collect_driver, time_step, policy_state = setup_rl_training_pipeline(
                tf_ppo_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Setup wandb logging
            artifact = initialize_wandb_logging(name=f"{csv_name}_Home{building_index}_rd{round}", num_iterations=num_iterations)
            
            #5. Train, evaluate agent and store weights
            result_df, metrics = agent_training_and_evaluation(global_step, num_test_iterations, collect_driver, 
                time_step, policy_state, iterator, tf_ppo_agent, eval_policy, building_index, result_df, eval_interval, environments)
            
            #6. End and log wandb
            end_and_log_wandb(metrics, artifact)

Cluster:  0  - Building:  7  - round:  0


c:\Users\rs1044\Documents\GitHub\Federated-Reinforcement-Learning-for-Battery-Charging\src\notebooks\..\utils\reinforcementLearningHelper.py:443: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat(


Building:  7  - Total Profit:  -793.5300297600002  - Total Emissions:  3916.97349463678
Cluster:  0  - Building:  7  - round:  1
Building:  7  - Total Profit:  -793.5300297600002  - Total Emissions:  3916.97349463678
Cluster:  0  - Building:  7  - round:  2
Building:  7  - Total Profit:  -793.5300297600002  - Total Emissions:  3916.97349463678
Cluster:  1  - Building:  3  - round:  0
Building:  3  - Total Profit:  -354.8583669999963  - Total Emissions:  1719.2886666884938
Cluster:  1  - Building:  3  - round:  1
Building:  3  - Total Profit:  -354.8583669999963  - Total Emissions:  1719.2886666884938
Cluster:  1  - Building:  3  - round:  2
Building:  3  - Total Profit:  -354.8583669999963  - Total Emissions:  1719.2886666884938
Cluster:  1  - Building:  4  - round:  0
Building:  4  - Total Profit:  -982.2749281299997  - Total Emissions:  4075.3946355054195
Cluster:  1  - Building:  4  - round:  1
Building:  4  - Total Profit:  -982.2749281299997  - Total Emissions:  4075.3946355054195

In [9]:
# Save results
result_df['Setup'] = csv_name
result_df.index.name = 'Building_nr'
result_df.reset_index(inplace=True, drop=True)
os.makedirs('results', exist_ok=True)
result_df.to_csv(f'results/{csv_name}.csv', index=False)

print("Final result: ", result_df["Total Profit"].sum())

Final result:  -107204.8854223708


In [10]:
import requests

def send_telegram_message(bot_token, chat_id, message):
    """Send a message to a Telegram chat via the Bot API."""
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    payload = {
        "chat_id": chat_id,
        "text": message,
        "parse_mode": "Markdown"
    }
    response = requests.post(url, json=payload)
    return response.json()

# Use the function
bot_token = os.getenv('TELEGRAM_BOT_TOKEN')
chat_id = os.getenv('TELEGRAM_CHAT_ID')
message = f"Script PPO FL ist fertig am PC .81!"

result = send_telegram_message(bot_token, chat_id, message)
print(result)

{'ok': True, 'result': {'message_id': 225, 'from': {'id': 7071194232, 'is_bot': True, 'first_name': 'Reinforcement Learning', 'username': 'FederatedRL_Bot'}, 'chat': {'id': 5493937056, 'first_name': 'Jonas', 'last_name': 'Sievers', 'username': 'JonasSievers', 'type': 'private'}, 'date': 1722041509, 'text': 'Script PPO FL ist fertig am PC .81!'}}


In [11]:
#import pandas as pd
#result_df = pd.read_csv("results\FL_DDPG_11Cluster_OnlyCosts_Figure7.csv")
result_df['Total Emissions'].sum()

431939.0371126558

In [12]:
std_dev = result_df.groupby('Building')['Total Emissions'].std().mean()
std_dev

0.05661883999438698